In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 4,
    "Parts": 5,
    "Warehouse": 6,
    "Supplier": 7,
}

In [3]:
API_URL = "http://localhost:8000"

version = "static_medium_100"

In [ ]:
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

print(create_ops)

In [ ]:
updated_create_ops = []

for op in create_ops:
    op["version"] = version 
    updated_create_ops.append(op)

requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_create_ops)

In [ ]:
update_cycles = 100
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    updated_update_ops = []

    for op in update_ops:
        op["version"] = version
        updated_update_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_update_ops)

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)